<a href="https://colab.research.google.com/github/NoPParkGGame/MyFirstMachineLearning/blob/main/GridSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

wine = pd.read_csv('http://bit.ly/wine_csv_data')

wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [5]:
data = wine[['alcohol', 'sugar', 'pH']].values

target = wine['class'].values

In [6]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [8]:
print(sub_input.shape, val_input.shape, test_input.shape)

(4157, 3) (1040, 3) (1300, 3)


In [46]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [48]:
# 교차검증 -> K - 폴드 교차검증
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)

print(scores)

{'fit_time': array([0.00967932, 0.00881338, 0.00965786, 0.00924659, 0.00859141]), 'score_time': array([0.00145197, 0.0020535 , 0.00182438, 0.00140119, 0.00142074]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [49]:
import numpy as np

# 교차검증 결과의 평균 -> 최상의 검증 점수
print(np.mean(scores['test_score']))


0.855300214703487


In [50]:
# 하이퍼파라미터 튜닝
# 1. 라이브러리가 제공하는 Default 값을 사용
# 2. 검증세트, 교차검증 -> 매개변수를 조금씩 바꿔가야 함
# 3. 매개변수는 다른 매개변수에 영향을 끼침 -> 일일히 노가다하며 찾는 것은 복잡함

In [51]:
# 그리드 서치 -> 하이퍼파라미터 탐색과 교차검증을 한 번에 수행

from sklearn.model_selection import GridSearchCV

# 탐색할 매개변수, 탐색할 값 딕셔너리
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [52]:
# GridSearchCV 는 점수가 가장 높은 모델의 매개변수 조합으로 best_estimator 모델을 만들어줌

dt = gs.best_estimator_

print(dt.score(train_input, train_target))

0.9615162593804117


In [54]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [55]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [56]:
print(gs.cv_results_['params'][gs.best_index_])

{'min_impurity_decrease': 0.0001}


In [57]:
# 정리
# 1. 탐색할 매개변수 지정, 값 지정
# 2. 훈련세트에서 그리드서치 수행 -> 최상의 평균검증점수가 나오는 매개변수 조합 -> 그리드서치 객체에 저장
# 3. 그 모델을 이용해 전체 훈련세트를 학습 -> 해당 모델도 서치 객체에 저장
# 좀 더 복잡하게는 할 수 없을까?

In [58]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)}
# 각 매개변수의 총 개수는 9, 15, 10개 -> 하나의 세트당 총 1350 번의 연산
# 5폴드 교차 -> 5 * 1350 -> 총 6750 번의 연산 -> 최상의 모델 찾아내기

gs = GridSearchCV(DecisionTreeClassifier(), params, n_jobs= -1)

gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [59]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}


In [61]:
print(max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [63]:
# 아까 지정한 매개변수의 간격을 랜덤하게 설정은 못할까?

In [68]:
# 싸이파이 라이브러리 이용
# uniform 실수 뽑기
# randint 정수 뽑기

from scipy.stats import uniform, randint

# 0~9 사이의 숫자를 생성하는 난수 생성기
rgen = randint(0, 10)

# rgen 값이 가지는 난수를 N개 생성해줘 하는 함수
rgen.rvs(10)

array([4, 5, 4, 4, 4, 2, 1, 4, 3, 3])

In [69]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([117,  91, 116,  82,  85,  96, 112, 103,  93, 105]))

In [71]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.0726286 , 0.9394034 , 0.50959919, 0.84173169, 0.59130018,
       0.21173869, 0.42902321, 0.62988794, 0.70839232, 0.91035642])

In [72]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20, 50),
          'min_samples_split' : (2, 25),
          'min_samples_leaf' : randint(1, 25)}

from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(DecisionTreeClassifier(random_state= 42), params, n_iter= 100, n_jobs= -1, random_state= 42)

rs.fit(train_input, train_target)


RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ce8aac75460>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ce8aac77590>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ce8ad5dcbf0>,
                                        'min_samples_split': (2, 25)},
                   random_state=42)

In [73]:
print(rs.best_params_)

{'max_depth': 46, 'min_impurity_decrease': np.float64(0.000517411003148779), 'min_samples_leaf': 5, 'min_samples_split': 2}


In [78]:
print(np.max(rs.cv_results_['mean_test_score']))
print(rs.best_score_)

0.8685788850225808
0.8685788850225808


In [79]:
dt = rs.best_estimator_

print(dt.score(test_input, test_target))

0.8584615384615385
